In [1]:
import numpy as np
from scipy import sparse
from scipy import stats
from scipy.optimize import minimize
from tqdm.auto import tqdm
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

from ray import train, tune
from ray.tune.search.bayesopt import BayesOptSearch

In [2]:
class RC:
    """

    A class which implements a reservoir computing framework for data forecasting.

    A default initialization creates a classical reservoir computer with a tanh input nonlinearity
    and a linear output map.
    x(t+1) = (1 - alpha)*x + alpha*f(Ax(t) + Bu(t) + b)
    y(t) = Cx(t)
    Attributes:
    -----------
    Nx: integer dimension of reservoir
    Nu: integer dimension of input
    f: input activation function; tanh unless otherwise defined
    A: Nx by Nx array, reservoir weights
    B: Nx by Nu array, input weights
    b: Nx by 1 array, bias
    sigma: positive scalar, range of values for B matrix [-sigma, sigma]
    alpha: positive scalar, time scale of reservoir
    rho_sr: positive scalar, spectral radius of reservoir (used to construct A matrix)
    sigma_b: positive scalar, strength of bias in reservoir
    rho_A: positive scalar in (0,1], density of connections in A
    beta: Tikhonov regularization parameter
    Forecasting: True or False, internal flag
    Training: True or False, internal flag
    x: Nx by 1 array, current reservoir state
    t: integer, current time step
    xt: Nx by Nt array, state history during training
    xf: Nx by Nf array, state history during forecasting
    p_form: string, form of input weights ("linear" or "quadratic")
    add_LS_bias: True or False, flag for adding bias to readout vector in least squares problem (default is False)

    Methods:
    -----------
    StepForward(u_k)
        Evolve the reservoir one time step forward
    Train_Traj(U, lambd = 1e-6, quadratic = False)
        Train the output weights given the time history of inputs
    Forecast_Traj(K)
        Forecast the time history of outputs for K time steps forward given the time history of inputs up through current time step
    build_A()
        Construct the reservoir weight matrix A
    build_B()
        Construct the input weight matrix B
    get_p()
        Construct the observation vector p used in forecasting and training

    Authors: Anastasia Bizyaeva, Jan Williams
    e-mail: anabiz@uw.edu

    """

    def __init__(self, Nx=200, Nu=3, f=np.tanh, A = 0, B = 0, sigma = 0.084, alpha = 0.6, rho_sr=0.80, sigma_b=1.6, rho_A=0.02, beta=8.493901e-8,leak =1.0, p_form = "linear", add_LS_bias = False,):

        """

        Initialize a ReservoirComputer instance.

            Attributes:
            -----------
            Nx: integer dimension of reservoir
            Nu: integer dimension of input
            f: input activatiion function; tanh unless otherwise defined
            g: output activation function; g(x) = x unless otherwise defined
            A: Nx by Nx array, reservoir weights; randomly generated if not provided
            B: Nx by Nu array, input weights; randomly generated if not provided
            b: Nx by 1 array, bias added inside reservoir
            alpha: positive scalar, time step of reservoir
            rho_sr: positive scalar, spectral radius of absolute value of reservoir (used to construct A matrix)
            sigma: positive scalar, range of input weights [-sigma,sigma] used to generate B matrix
            sigma_b: positive scalar, scaling factor of bias (used to construct b vector)
            rho_A: scalar in [0,1] interval, probability of connection between reservoir nodes
            Forecasting: True or False, internal flag
            Training: True or False, internal flag
            x: Nx by 1 array, current reservoir state
            t: integer, current time step
            xt: Nx by Nt array, reservoir state history during training (initiated with initial condition)
            xf: Nx by Nf array, reservoir state history during forecasting (initiated empty)
            p_form: string, form of input weights ("linear" or "quadratic") - default "linear"
            add_LS_bias: True or False, flag for adding a random bias to readout vector in least squares (default is False)
            leak: leak rate of reservoir
        """

        self.Nx = Nx
        self.Nu = Nu
        self.f = f
        self.sigma = sigma
        self.alpha = alpha
        self.rho_sr = rho_sr
        self.sigma_b = sigma_b
        self.rho_A = rho_A
        self.beta = beta
        self.Forecasting = False
        self.Training = False
        self.p_form = p_form
        self.add_LS_bias = add_LS_bias
        self.x = np.zeros([self.Nx,1])
        self.leak=leak

        if isinstance(A,int):            # reservoir weights (initialized randomly when not user-supplied)
            # construct A matrix

            # generate sparse random matrix of size Nx by Nx with connection density rho_A
            A = self.build_A()

            # check if A is full rank, if not repeat until it is
            while_count = 0
            while np.linalg.matrix_rank(A) < Nx:
                A = self.build_A()
                while_count+=1
                if while_count > 10: # if A is not full rank after 10 tries, break and use A as is
                    print('Warning: A matrix is not full rank')
                    break
            self.A = A

        else:
            # A matrix is user-supplied

            # Check that A is a square matrix of size Nx by Nx
            if A.shape[0] != Nx or A.shape[1] != Nx:
                raise ValueError("A is not a square matrix of size Nx by Nx")
            else:
                self.A = A
        # input weights (initialized randomly when not user-supplied)
        # initialize B to be a random dense matrix of size Nx by Nu with uniformly distributed entries between -sigma and sigma
        if isinstance(B,int):

            B = self.build_B()
            self.B = B

        else:
            # Check that B is a square matrix of size Nx by Nu
            if B.shape[0] != Nx or B.shape[1] != Nu:
                raise ValueError("B is not a square matrix of size Nx by Nu")
            else:
                self.B = B

        self.t = 0                                          # initial time step
        #self.xt =np.array(self.x)                           # state history during training: add initial condition as first row of array
        #self.xf = np.empty((self.Nx,1),dtype=np.float64)    # state history during forecasting

        self.b = self.sigma_b * np.ones([self.Nx,1])        # construct bias vector

        self.train_counter = 0
        self.forecast_counter = 0


    def build_A(self):
        """
        Build a sparse random matrix A with connection density rho_A and spectral radius rho_sr
        Outputs:
        -----------
        A: Nx by Nx numpy array, sparse random matrix with density rho_A and spectral radius rho_sr
        """
        A = np.random.uniform(-1,1,size=self.Nx ** 2)
        del_indices = np.random.choice(self.Nx ** 2, size=int((1-self.rho_A)*self.Nx ** 2),replace=False)
        A[del_indices] = 0
        A = A.reshape(self.Nx, self.Nx)
        A = A*(self.rho_sr/np.max(np.abs(np.linalg.eigvals(A))))

        return A

    def build_B(self):
        """
        Build an input matrix B with entries chosen uniformly [-sigma, sigma]
        Outputs:
        -----------
        B: Nx by Nu numpy array, random input matrix mapping from dim(Nu) to dim(Nx)
        """
        B = np.random.uniform(-self.sigma, self.sigma, size=(self.Nx, self.Nu))
        return B


    def StepForward(self,u_k, i=None):
        """
        Advance the reservoir state one time step according to x(t+1) = (1-alpha)x(t) + alpha*tanh(Ax(t) + Bu(t) + b)
            Inputs:
            -----------
            u_k: Nu by 1 vector of inputs to the reservoir at the current time step
        """
        if u_k.shape[0] != self.Nu:
            raise ValueError("Input u_k is not a vector of length Nu")
        else:
            if u_k.ndim == 1:
                u_k = u_k.reshape(self.Nu,1) # convert u_k to a column vector

            self.x = (1 - self.alpha*self.leak)*self.x + self.alpha * self.f(self.A.dot(self.x) + self.B.dot(u_k) + self.b)         # update reservoir state taking one step forward

            if self.Training:
                self.xt[:, self.train_counter:1+self.train_counter] = self.x
                self.train_counter += 1
                #self.xt = np.concatenate((self.xt,self.x),axis=1)         # add current reservoir state to training history
            if self.Forecasting:
                self.xf[:, self.forecast_counter:1+self.forecast_counter] = self.x
                self.forecast_counter += 1
                #self.xf = np.concatenate((self.xf,self.x),axis=1)         # add current reservoir state to forecasting history
            self.t +=1

    def get_p(self,x_t):
        """

        Compute the parameter vector used in least squares problem in reservoir training and in forecasting

            Inputs:
            -----------
            x_t: either Nt by Nx array(reservoir state history) or Nx-element 1d array (reservoir state at a particular time step)
            Returns:
            -----------
            p: numpy array, parameter vector used in least squares problem in reservoir training and in forecasting

        """
        # check that input is a 2d array
        if x_t.ndim != 2:
            raise ValueError("Input must be a 2d array")


        Nt = x_t.shape[1] # number of time steps

        if self.p_form == "linear":
            p = x_t
        if self.p_form == "quadratic":
            p = np.concatenate((x_t,np.square(x_t)),axis = 0)
        if self.add_LS_bias:
            p = np.concatenate((np.ones([1,Nt]),p),axis = 0)

        return p



    def Train_Traj(self,U, spinup=500,verbose=False):
        """

        Train the output weights given the time history of inputs up through current time step

            Inputs:
            -----------
            U: 2darray with dimension Nu by Nt, time history of input for training
            lambd: regularization coefficient for pseudoinverse; prevents ill-conditioning of weights from overfitting

        """
        if U.shape[0] != self.Nu:
            raise ValueError("U is not a matrix of size Nu by number of training time steps")
        else:
            for i in range(spinup):
                self.StepForward(U[:,i])

            self.Training = True

            self.xt = np.zeros([self.Nx, len(U[0,spinup+1:])])
            # assert self.xt.shape[1] == 14499, "shape should in 14500"

            if verbose == False:
                for i in range(spinup,U.shape[1] - 1):
                    self.StepForward(U[:,i])
                    if np.linalg.norm(self.x) > 1e10 or np.isnan(np.linalg.norm(self.x)):
                        print('Reservoir trajectory blew up after '+str(i)+' steps in training')
                        self.Training = False # turn off training flag
                        break
            if verbose == True:
                clear_output(wait=True)
                for i in tqdm(range(spinup,U.shape[1] - 1)):
                    self.StepForward(U[:,i])
                    if np.linalg.norm(self.x) > 1e10 or np.isnan(np.linalg.norm(self.x)):
                        print('Reservoir trajectory blew up after '+str(i)+' steps in training')
                        self.Training = False # turn off training flag
                        break
            
            x_mat = self.get_p(self.xt)
            u_mat = U[:, spinup+1:]
            
            '''
            self.C0 = []
            self.xt0 = []
            for i in range(int(trainl/step)):
                x_matn = x_mat[:, 0:(i+1)*step]
                u_matn = U[:, spinup+1:spinup+1+(i+1)*step]
                lhs = (x_matn.dot(x_matn.T) + self.beta * np.eye(self.Nx))
                rhs = x_matn.dot(u_matn.T)
                C_mat_Tn = np.linalg.lstsq(lhs, rhs, rcond=None)[0]
                self.C0.append(C_mat_Tn.T)
                self.xt0.append(self.xt)
            '''
            
            # train output weights using regularized least squares            
            lhs = (x_mat.dot(x_mat.T) + self.beta * np.eye(self.Nx))
            rhs = x_mat.dot(u_mat.T)
            C_mat_T = np.linalg.lstsq(lhs, rhs, rcond=None)[0]
            self.C = C_mat_T.T

            # print(np.sum((self.C0[-1] - self.C)**2))

        self.Training = False

    def Forecast_Traj(self,K, spinup_data = None,verbose=False):
        """

        Assuming reservoir has been trained, forecast the trajectory forward in time

            Inputs:
            -----------
            K: integer, number of time steps for forward forecasting

        """

        ### forecast from the existing internal reservoir state x
        if spinup_data is not None:
            self.Training = False
            self.Forecasting = False
            self.x = np.zeros([self.Nx,1])
            for i in range(spinup_data.shape[1]):
                self.StepForward(spinup_data[:,i])

        self.Forecasting = True
        self.forecast_counter = 0
        self.xf = np.zeros([self.Nx, K])
        if verbose == False:
            for i in range(K):
                p = self.get_p(self.x)
                temp_u = self.C.dot(p)
                self.StepForward(temp_u)
                if np.linalg.norm(self.x) > 1e10 or np.isnan(np.linalg.norm(self.x)):
                        print('Forecasted trajectory blew up after '+str(i)+' steps')
                        self.Forecasting = False # turn off forecasting flag
                        break
        if verbose == True:
            for i in tqdm(range(K)):
                p = self.get_p(self.x)
                temp_u = self.C.dot(p)
                self.StepForward(temp_u)
                if np.linalg.norm(self.x) > 1e10 or np.isnan(np.linalg.norm(self.x)):
                        print('Forecasted trajectory blew up after '+str(i)+' steps')
                        self.Forecasting = False # turn off forecasting flag
                        break

        self.Forecasting = False


### TODO: Investigate computational speedups/parallelization for the optimization routine

In [3]:
# Lorenz 63
def Lorenz(t, x, sigma=10, beta=8/3, rho=28):
    return np.array([sigma * (x[1] - x[0]),
            x[0] * (rho - x[2]) - x[1],
            x[0] * x[1] - beta * x[2]])

# Blasius
def Blasius(t, x, a = 1, alpha1 = 0.2, alpha2 = 1, b = 1, c = 10, k1 = 0.05, k2 = 0, zs = 0.006):
    xdot = a * x[0] - alpha1 * x[0] * x[1] / (1 + k1 * x[0])
    ydot = -b * x[1] + alpha1 * x[0] * x[1] / (1 + k1 * x[0]) - alpha2 * x[1] * x[2] / (1 + k2 * x[1])
    zdot = -c * (x[2] - zs) + alpha2 * x[1] * x[2] / (1 + k2 * x[1])
    return np.array([xdot, ydot, zdot])

# Dadras
def Dadras(t, x, c=2, e=9, o=2.7, p=3, r=1.7):
    xdot = x[1] - p * x[0] + o * x[1] * x[2]
    ydot = r * x[1] - x[0] * x[2] + x[2]
    zdot = c * x[0] * x[1] - e * x[2]
    return np.array([xdot, ydot, zdot])



In [11]:
T = 600

x0 = np.array([4.031713, 5.1113788, 0.016508812])
perturbations = np.random.normal(loc=0, scale=0.1, size=3)
x0 = x0 + perturbations

dt_data = 0.01  # sampling time step
xt = integrate.solve_ivp(Blasius, [0, T], x0, method='RK45', t_eval=np.arange(0, T, dt_data), rtol=1e-12)
U = xt.y  # training signal
print(U.shape)

(3, 60000)


In [20]:
def train_esn(config):
    '''Nx: integer dimension of reservoir
    Nu: integer dimension of input
    f: input activatiion function; tanh unless otherwise defined
    A: Nx by Nx array, reservoir weights
    B: Nx by Nu array, input weights
    b: Nx by 1 array, bias
    sigma: positive scalar, range of values for B matrix [-sigma, sigma]
    alpha: positive scalar, leak rate of reservoir
    rho_sr: positive scalar, spectral radius of reservoir (used to construct A matrix)
    sigma_b: positive scalar, strength of bias in reservoir
    rho_A: positive scalar in (0,1], density of connections in A'''

    Nx=200; Nu=3; f=np.tanh;

    # Extract the hyperparameters from the config
    # Nx = int(config["Nx"]) # integer dimension of reservoir
    sigma = config["sigma"] # range of values for B matrix [-sigma, sigma]
    alpha = config["alpha"] # leak rate of reservoir
    rho_sr = config["rho_sr"] # spectral radius of reservoir
    sigma_b = config["sigma_b"] # strength of bias in reservoir
    rho_A = config["rho_A"] # density of connections in A
    beta = config["beta"] # Tikhonov regularization parameter

    # Initialize the ESN with the hyperparameters
    rc = RC(Nx=Nx, Nu=3, f=f, A=0, B=0, sigma=sigma, alpha=alpha, rho_sr=rho_sr, sigma_b=sigma_b, rho_A=rho_A, beta=beta, p_form = "linear", add_LS_bias = False)


    # Train the ESN on the training data
    train_len = 5500 # number of samples to use in training
    sp = 500 # number of transient points to discard at the beginning
    rc.Train_Traj(U[:,:train_len], spinup = sp)

    # Test the ESN on the test data
    forecast_len = 10000 # number of time steps to forecast
    rc.Forecast_Traj(forecast_len)
    forecasted_traj = rc.C.dot(rc.get_p(rc.xf))

    # Compute the performance metric (e.g., mean squared error)
    lambda_1 = 0.9
    lyap_t = 1/lambda_1
    lyap_n = int(lyap_t/dt_data)
    t = np.linspace(0, forecast_len*dt_data, forecast_len) * lambda_1
    print(t.shape)
    
    samp_n = 50
    error = np.zeros(samp_n)
    step = int((forecast_len-lyap_n)/samp_n)

    for i in range(samp_n):
        error[i] = np.sqrt(np.sum(abs(U[0, train_len+i*step:train_len+i*step+lyap_n] - forecasted_traj[0, i*step:i*step+lyap_n])**2)*(t[i*step+lyap_n]-t[i*step]))
    score = np.mean(error)

    # Report the MSE to Ray Tune
    train.report({"mean_loss": score})


# Define the search space for hyperparameters
search_space = {
    # "Nx": tune.uniform(50, 500),
    "sigma": tune.uniform(0.01, 0.5),
    "alpha": tune.uniform(0.0, 0.5),
    "rho_sr": tune.uniform(0.0, 1),
    "sigma_b": tune.uniform(1, 2),
    "rho_A": tune.uniform(0.0, 1),
    "beta": tune.uniform(1e-6, 0.0001)
}

# Configure the Bayesian Optimization search algorithm
search_alg = BayesOptSearch(
    space=search_space,
    metric="mean_loss",
    mode="min",
    random_search_steps=5  # Number of random steps before using Bayesian optimization
)

# Run the Ray Tune hyperparameter search with Bayesian Optimization
analysis = tune.run(
    train_esn,
    search_alg=search_alg,
    num_samples=500,  # Number of hyperparameter sets to evaluate
    verbose=1
)

# Get the best hyperparameters
best_config = analysis.get_best_config(metric="mean_loss", mode="min")
print("Best hyperparameters found were: ", best_config)

# Nx=200; Nu=3; f=np.tanh; sigma=0.084; alpha=0.1; rho_sr=0.8; sigma_b=1.6; rho_A=0.1; beta=8.493901e-8;

# 0.253183	2.06029e-05	0.877276	0.605324	0.478606	1.98323	2.99262
# 0.380072	2.7796e-05	0.289138	0.326749	0.44264	1.89319	2.49282
# 0.0711564	1e-06	0.984754	0.95089	0.277254	1.47935	2.53347

(train_esn pid=226372) (10000,)
(train_esn pid=226447) (10000,)
(train_esn pid=226530) (10000,)
(train_esn pid=226614) (10000,)
(train_esn pid=226695) (10000,)
(train_esn pid=226786) (10000,)
(train_esn pid=226867) (10000,)
(train_esn pid=226950) (10000,)
(train_esn pid=227035) (10000,)
(train_esn pid=227110) (10000,)
(train_esn pid=227200) (10000,)
(train_esn pid=227283) (10000,)
(train_esn pid=227365) (10000,)
(train_esn pid=227448) (10000,)
(train_esn pid=227531) (10000,)
(train_esn pid=227613) (10000,)
(train_esn pid=227698) (10000,)
(train_esn pid=227781) (10000,)
(train_esn pid=227863) (10000,)
(train_esn pid=227947) (10000,)
(train_esn pid=228030) (10000,)
(train_esn pid=228116) (10000,)
(train_esn pid=228197) (10000,)
(train_esn pid=228280) (10000,)
(train_esn pid=228362) (10000,)
(train_esn pid=228445) (10000,)
(train_esn pid=228528) (10000,)
(train_esn pid=228610) (10000,)
(train_esn pid=228693) (10000,)
(train_esn pid=228775) (10000,)
(train_esn pid=228859) (10000,)
(train_e

(train_esn pid=231078) /tmp/ipykernel_59016/1652708507.py:159: RuntimeWarning: divide by zero encountered in scalar divide
(train_esn pid=231078) /tmp/ipykernel_59016/1652708507.py:159: RuntimeWarning: invalid value encountered in multiply
2024-08-29 17:33:57,636	ERROR tune_controller.py:1331 -- Trial task failed for trial train_esn_265db934
Traceback (most recent call last):
  File "/home/daveli/anaconda3/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/daveli/anaconda3/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/daveli/anaconda3/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/daveli/anaconda3/lib/python3.11/site-packages/r

(train_esn pid=231153) (10000,)


2024-08-29 17:34:00,351	ERROR tune_controller.py:1331 -- Trial task failed for trial train_esn_8654f65d
Traceback (most recent call last):
  File "/home/daveli/anaconda3/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/daveli/anaconda3/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/daveli/anaconda3/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/daveli/anaconda3/lib/python3.11/site-packages/ray/_private/worker.py", line 2661, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/daveli/anaconda3/l

(train_esn pid=231324) (10000,)


2024-08-29 17:34:03,184	INFO bayesopt_search.py:293 -- Skipping duplicated config: {'alpha': 0.0, 'beta': 1e-06, 'rho_A': 0.0, 'rho_sr': 1.0, 'sigma': 0.5, 'sigma_b': 1.0}.


(train_esn pid=231407) (10000,)
(train_esn pid=231489) (10000,)


2024-08-29 17:34:06,203	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/daveli/ray_results/train_esn_2024-08-29_17-32-40' in 0.0273s.


(train_esn pid=231572) (10000,)


TuneError: ('Trials did not complete', [train_esn_265db934, train_esn_8654f65d])

In [ ]:
def train_esn(config):
    '''Nx: integer dimension of reservoir
    Nu: integer dimension of input
    f: input activatiion function; tanh unless otherwise defined
    A: Nx by Nx array, reservoir weights
    B: Nx by Nu array, input weights
    b: Nx by 1 array, bias
    sigma: positive scalar, range of values for B matrix [-sigma, sigma]
    alpha: positive scalar, leak rate of reservoir
    rho_sr: positive scalar, spectral radius of reservoir (used to construct A matrix)
    sigma_b: positive scalar, strength of bias in reservoir
    rho_A: positive scalar in (0,1], density of connections in A'''

    Nx=200; Nu=3; f=np.tanh;

    # Extract the hyperparameters from the config
    # Nx = int(config["Nx"]) # integer dimension of reservoir
    sigma = config["sigma"] # range of values for B matrix [-sigma, sigma]
    alpha = config["alpha"] # leak rate of reservoir
    rho_sr = config["rho_sr"] # spectral radius of reservoir
    sigma_b = config["sigma_b"] # strength of bias in reservoir
    rho_A = config["rho_A"] # density of connections in A
    beta = config["beta"] # Tikhonov regularization parameter

    errorM = np.zeros(15)

    for M in range(15):

        # Initialize the ESN with the hyperparameters
        rc = RC(Nx=Nx, Nu=3, f=f, A=0, B=0, sigma=sigma, alpha=alpha, rho_sr=rho_sr, sigma_b=sigma_b, rho_A=rho_A, beta=beta, p_form = "linear", add_LS_bias = False)

        # Generate training data
        T = 600
        dt_data = 0.01

        x0 = np.array([-0.1, 1, 1.05])
        perturbations = np.random.normal(loc=0, scale=0.1, size=3)
        x0 = x0 + perturbations

        xt = integrate.solve_ivp(Lorenz, [0, T], x0, method='RK45', t_eval=np.arange(0, T, dt_data),rtol=1e-12)
        U = xt.y

        # Train the ESN on the training data
        train_len = 14500 # number of samples to use in training
        sp = 500 # number of transient points to discard at the beginning
        rc.Train_Traj(U[:,:train_len], spinup = sp)

        # Test the ESN on the test data
        forecast_len = 2000 # number of time steps to forecast
        rc.Forecast_Traj(forecast_len)
        forecasted_traj = rc.C.dot(rc.get_p(rc.xf))

        # Compute the performance metric (e.g., mean squared error)
        lambda_1 = 0.9
        t = np.linspace(0, forecast_len*dt_data, forecast_len) * lambda_1
        print()
        errorM[M] = np.sum(abs(U[0, train_len:train_len+forecast_len] - forecasted_traj[0, :])**2*np.exp(-t/t[-1]))
        print(errorM)

    score = np.sum(errorM)

    # Report the MSE to Ray Tune
    train.report({"mean_loss": score})


# Define the search space for hyperparameters
search_space = {# Nx=200; Nu=3; f=np.tanh; sigma=0.084; alpha=0.1; rho_sr=0.8; sigma_b=1.6; rho_A=0.1; beta=8.493901e-8;
    "sigma": tune.uniform(0.1, 1),
    "alpha": tune.uniform(0.0, 0.5),
    "rho_sr": tune.uniform(0.01, 1.0),
    "sigma_b": tune.uniform(0.0, 2),
    "rho_A": tune.uniform(0.0, 1.0),
    "beta": tune.uniform(10e-10, 10e-5)
}

# Configure the Bayesian Optimization search algorithm
search_alg = BayesOptSearch(
    space=search_space,
    metric="mean_loss",
    mode="min",
    random_search_steps=5  # Number of random steps before using Bayesian optimization
)

# Run the Ray Tune hyperparameter search with Bayesian Optimization
analysis = tune.run(
    train_esn,
    search_alg=search_alg,
    num_samples=100,  # Number of hyperparameter sets to evaluate
    verbose=1
)

# Get the best hyperparameters
best_config = analysis.get_best_config(metric="mean_loss", mode="min")
print("Best hyperparameters found were: ", best_config)

(train_esn pid=280014) 
(train_esn pid=280014) [143138.04811189      0.              0.              0.
(train_esn pid=280014)       0.              0.              0.              0.
(train_esn pid=280014)       0.              0.              0.              0.
(train_esn pid=280014)       0.              0.              0.        ]
(train_esn pid=280123) 
(train_esn pid=280123) [91965.34348725     0.             0.             0.
(train_esn pid=280123)      0.             0.             0.             0.
(train_esn pid=280123)      0.             0.             0.             0.
(train_esn pid=280352) 
(train_esn pid=280352) [184772.75355197      0.              0.              0.
(train_esn pid=280352)       0.              0.              0.              0.
(train_esn pid=280352)       0.              0.              0.              0.
(train_esn pid=280352)       0.              0.              0.        ] [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DE